**Social Vulnerability Data**

Jacqueline Simeone

In [2]:
import pandas as pd
import numpy as np

In [3]:
df_og = pd.read_csv('../source_data/SRC_SVI2018_US.csv')

In [4]:
#according to cdc website, nans are listed as -999.0
df_og.replace(-999.0,np.nan, inplace=True)

In [5]:
df_og.isnull().sum().sort_values(ascending=False).head(1)

F_TOTAL    664
dtype: int64

In [6]:
#create county_state feature
df_og.rename(columns={'COUNTY':'og_county'}, inplace=True)
df_og['county']=df_og['og_county']+'_'+df_og['ST_ABBR']

In [7]:
themes_all=list(df_og['county'])
themes_null=list(df_og[df_og['RPL_THEMES'].isnull()]['county'])
themes_notnull = list(df_og[~df_og['RPL_THEMES'].isnull()]['county'])

nulls=[]
[nulls.append(x) for x in themes_null if x not in nulls]

alls=[]
[alls.append(x) for x in themes_all if x not in alls]

print(len(alls))
print(len(nulls))

3142
340


In [8]:
df_og.isnull().sum().sort_values(ascending=False).head(2)

F_TOTAL       664
SPL_THEMES    664
dtype: int64

In [9]:
#let's dummy whether or not the rpl themes column had a null.
#then find the percent of nulls (% of tracks with missing data in counties)
#creating new dataframe with this info to then merge at the county index
df_og['RPL_THEMES_DUM']=(df_og['RPL_THEMES'].isnull()).astype(int)

In [10]:
df_rpl_dum = pd.DataFrame(df_og.copy().groupby('county')['RPL_THEMES_DUM'].mean())

In [11]:
#making a second county column so that I can use groupby after idexing
df_og['county_2'] = df_og['county']

In [12]:
#index on county
df_og.set_index('county', inplace=True)

In [13]:
df = pd.merge(df_og,df_rpl_dum, left_index=True, right_index=True)
df.rename(columns={'RPL_THEMES_DUM_x':'RPL_THEMES_DUM', 'RPL_THEMES_DUM_y':'RPL_THEMES_DUM_PCT'}, inplace=True)

Looked into whetehr or not dropping nans made sense above.
Ended up making sense, as only one county was removed, Rio Arriba_NM. I'm going to manually add parameters for Rio Arriba.

In [14]:
rio = df_og[df_og['og_county']=='Rio Arriba'][['E_UNINSUR','E_TOTPOP']]
rio['weight']=rio['E_TOTPOP']/39307

In [15]:
rio['UNINSURED_COUNT']=rio['E_UNINSUR']*rio['weight']

In [16]:
#total percent uninsured
rio['UNINSURED_COUNT'].sum()/39307

0.012863340623561969

In [17]:
df_rio = df_og[df_og['county_2']=='Rio Arriba_NM']

In [18]:
#deciding to drop nans because we don't lose any counties by dropping
#when we drop all, we lose 664 rows #df_og.shape[0]-df.shape[0]
#this is equal to the amount of nans in the important summary stats
df = df.copy().dropna()

In [19]:
#double check we still have 3142 counties
#list on county_2 because new index is 'county'
themes_all=list(df['county_2'])

all_lst=[]
[all_lst.append(x) for x in themes_all if x not in all_lst]

print(len(all_lst))

3141


In [20]:
len(all_lst)

3141

In [21]:
len(alls)

3142

In [22]:
#one county was dropped. trying to see intersection but nothing is showing.
def difference(lst1, lst2):
   # final_list = list(set().difference(lst1, lst2))
    final_list = set(lst1).difference(set(lst2))
    return final_list

difference(alls,all_lst)

{'Rio Arriba_NM'}

create count variable. if count != 1, then proceed with std column

In [23]:
df_count = pd.DataFrame(df.groupby('county_2')['ST'].count())
df_count.rename(columns={'ST':'COUNT'}, inplace=True)
df = pd.merge(df,df_count, left_index=True, right_index=True)

Add SD of 'RPL_THEME1', 'RPL_THEME2','RPL_THEME3','RPL_THEME4','RPL_THEMES'

In [24]:
def sd(column):
    df_new = pd.DataFrame(df.groupby('county_2')[f'{column}'].describe())
    df_new = df_new[['std']]
    df = pd.merge(df,df_new, left_index=True, right_index=True)
    df.rename(columns={'std':f'{column}_STD'}, inplace=True)

In [25]:
#['RPL_THEME1', 'RPL_THEME2','RPL_THEME3','RPL_THEME4','RPL_THEMES']:
df_new = pd.DataFrame(df.groupby('county_2')['RPL_THEME1'].describe())
df_new = df_new[['std']]
df = pd.merge(df,df_new, left_index=True, right_index=True)
df.rename(columns={'std':'RPL_THEME1_STD'}, inplace=True)

In [26]:
#left: 'RPL_THEME2','RPL_THEME3','RPL_THEME4','RPL_THEMES']
df_new = pd.DataFrame(df.groupby('county_2')['RPL_THEME2'].describe())
df_new = df_new[['std']]
df = pd.merge(df,df_new, left_index=True, right_index=True)
df.rename(columns={'std':'RPL_THEME2_STD'}, inplace=True)

In [27]:
#left: 'RPL_THEME3','RPL_THEME4','RPL_THEMES']
df_new = pd.DataFrame(df.groupby('county_2')['RPL_THEME3'].describe())
df_new = df_new[['std']]
df = pd.merge(df,df_new, left_index=True, right_index=True)
df.rename(columns={'std':'RPL_THEME3_STD'}, inplace=True)

In [28]:
#left: 'RPL_THEME4','RPL_THEMES']
df_new = pd.DataFrame(df.groupby('county_2')['RPL_THEME4'].describe())
df_new = df_new[['std']]
df = pd.merge(df,df_new, left_index=True, right_index=True)
df.rename(columns={'std':'RPL_THEME4_STD'}, inplace=True)

In [29]:
#left:'RPL_THEMES'
df_new = pd.DataFrame(df.groupby('county_2')['RPL_THEMES'].describe())
df_new = df_new[['std']]
df = pd.merge(df,df_new, left_index=True, right_index=True)
df.rename(columns={'std':'RPL_THEMES_STD'}, inplace=True)

Now that we have STDs and no longer need repeating counties, let's find percent populations per entry. we'll then weight columns we want to keep.

Deciding which columns to keep. 
As counties have multiple entries, I'll have to add SD, weighted scores, and county count.

In [30]:
#add total county population and then new column for % of county pop
df_TOTPOP = pd.DataFrame(df.groupby('county_2')['E_TOTPOP'].sum())
df_TOTPOP.rename(columns={'E_TOTPOP':'POP_TOTAL'},index={'count_2': 'county'}, inplace=True)
df = pd.merge(df,df_TOTPOP, left_index=True, right_index=True)

In [31]:
#create column 'PCT_POP'
df['POP_PCT']=df['E_TOTPOP']/df['POP_TOTAL']

In [32]:
#'E_UNINSUR' is the number of uninsured people. it should be added up then divided by total pop
df_uninsured = pd.DataFrame(df.groupby('county_2')['E_UNINSUR'].sum())
df_uninsured.rename(columns={'E_UNINSUR':'E_UNINSUR_TOTAL'},index={'count_2': 'county'}, inplace=True)
df = pd.merge(df,df_uninsured, left_index=True, right_index=True)

#divide total uninsured by population total
df['PCT_UNINSURED'] = df['E_UNINSUR_TOTAL']/df['POP_TOTAL']

In [33]:
#multiply 'RPL_THEME1', 'RPL_THEME2','RPL_THEME3','RPL_THEME4','RPL_THEMES' by 'POP_PCT'
#then sum for final

df['RPL_THEME1_WEIGHTED']= df['RPL_THEME1']*df['POP_PCT']
df['RPL_THEME2_WEIGHTED'] = df['RPL_THEME2']*df['POP_PCT']
df['RPL_THEME3_WEIGHTED'] = df['RPL_THEME3']*df['POP_PCT']
df['RPL_THEME4_WEIGHTED'] = df['RPL_THEME4']*df['POP_PCT']
df['RPL_THEMES_WEIGHTED'] = df['RPL_THEMES']*df['POP_PCT']

In [34]:
#create and add weighted rpl theme columns to df

df_theme1_weight = pd.DataFrame(df.groupby('county_2')['RPL_THEME1_WEIGHTED'].sum())
df_theme2_weight = pd.DataFrame(df.groupby('county_2')['RPL_THEME2_WEIGHTED'].sum())
df_theme3_weight = pd.DataFrame(df.groupby('county_2')['RPL_THEME3_WEIGHTED'].sum())
df_theme4_weight = pd.DataFrame(df.groupby('county_2')['RPL_THEME4_WEIGHTED'].sum())
df_themes_weight = pd.DataFrame(df.groupby('county_2')['RPL_THEMES_WEIGHTED'].sum())

df = pd.merge(df,df_theme1_weight, left_index=True, right_index=True)
df.rename(columns={'RPL_THEME1_WEIGHTED_y':'RPL1_WEIGHTED_SUM', 
                   'RPL_THEME1_WEIGHTED_x':'RPL1_WEIGHTED_INDIV'}, inplace=True)
df = pd.merge(df,df_theme2_weight, left_index=True, right_index=True)
df.rename(columns={'RPL_THEME2_WEIGHTED_y':'RPL2_WEIGHTED_SUM', 
                   'RPL_THEME2_WEIGHTED_x':'RPL2_WEIGHTED_INDIV'}, inplace=True)
df = pd.merge(df,df_theme3_weight, left_index=True, right_index=True)
df.rename(columns={'RPL_THEME3_WEIGHTED_y':'RPL3_WEIGHTED_SUM', 
                   'RPL_THEME3_WEIGHTED_x':'RPL3_WEIGHTED_INDIV'}, inplace=True)
df = pd.merge(df,df_theme4_weight, left_index=True, right_index=True)
df.rename(columns={'RPL_THEME4_WEIGHTED_y':'RPL4_WEIGHTED_SUM', 
                   'RPL_THEME4_WEIGHTED_x':'RPL4_WEIGHTED_INDIV'}, inplace=True)
df = pd.merge(df,df_themes_weight, left_index=True, right_index=True)
df.rename(columns={'RPL_THEMES_WEIGHTED_y':'RPLS_WEIGHTED_SUM', 
                   'RPL_THEMES_WEIGHTED_x':'RPLS_WEIGHTED_INDIV'}, inplace=True)

In [46]:
columns_keep = [ 
                'RPL_THEME1_STD', 'RPL_THEME2_STD','RPL_THEME3_STD',
                'RPL_THEME4_STD','RPL_THEMES_STD',
                #'POP_PCT', 'POP_TOTAL', 'RPL_THEMES_DUM', 'RPL3_WEIGHTED_INDIV'
                # 'RPL1_WEIGHTED_INDIV','RPL2_WEIGHTED_INDIV','RPL4_WEIGHTED_INDIV',
                #  'RPLS_WEIGHTED_INDIV'
                'RPL_THEMES_DUM_PCT', 'PCT_UNINSURED', 'RPL1_WEIGHTED_SUM',
                'RPL2_WEIGHTED_SUM', 'RPL3_WEIGHTED_SUM', 'RPL4_WEIGHTED_SUM',
                'RPLS_WEIGHTED_SUM', 'COUNT','FIPS'
               ]

df[columns_keep].head(2)

,RPL_THEME1_STD,RPL_THEME2_STD,RPL_THEME3_STD,RPL_THEME4_STD,RPL_THEMES_STD,RPL_THEMES_DUM_PCT,PCT_UNINSURED,RPL1_WEIGHTED_SUM,RPL2_WEIGHTED_SUM,RPL3_WEIGHTED_SUM,RPL4_WEIGHTED_SUM,RPLS_WEIGHTED_SUM,COUNT,FIPS
Abbeville_SC,0.089387,0.195687,0.160383,0.134782,0.13034,0.0,0.11514,0.799646,0.793354,0.352214,0.674209,0.752302,6,45001950400
Abbeville_SC,0.089387,0.195687,0.160383,0.134782,0.13034,0.0,0.11514,0.799646,0.793354,0.352214,0.674209,0.752302,6,45001950600


In [47]:
df[columns_keep].drop_duplicates().shape[0]

72173

In [57]:
df_clean = df[columns_keep].drop_duplicates()

In [58]:
df_clean.rename(columns={'RPL_THEME1_STD':'socio_std',
                         'RPL_THEME2_STD':'hh_std',
                         'RPL_THEME3_STD':'minority_std',
                         'RPL_THEME4_STD':'housing_std',
                         'RPL_THEMES_STD':'overall_std',
                         'RPL_THEMES_DUM_PCT':'null_pct',
                         'PCT_UNINSURED':'uninsured_pct',
                         'RPL1_WEIGHTED_SUM':'socio_wgt',
                         'RPL2_WEIGHTED_SUM':'hh_wgt',
                         'RPL3_WEIGHTED_SUM':'minority_wgt',
                         'RPL4_WEIGHTED_SUM':'housing_wgt',
                         'RPLS_WEIGHTED_SUM':'overall_wgt',
                         'COUNT':'count'
                        }, inplace=True)

In [40]:
#df_og[df_og['STATE']=='NEW MEXICO']#.value_counts().head(30)

In [59]:
df_clean.reset_index(inplace=True)

In [60]:
df_clean['index']

0        Abbeville_SC
1        Abbeville_SC
2        Abbeville_SC
3        Abbeville_SC
4        Abbeville_SC
             ...     
72168       Zavala_TX
72169       Zavala_TX
72170       Zavala_TX
72171       Zavala_TX
72172      Ziebach_SD
Name: index, Length: 72173, dtype: object

In [55]:
df_clean[df_clean['index']=='Estancia_NM']
#039 rio 055 taos

,index,socio_std,hh_std,minority_std,housing_std,overall_std,null_pct,uninsured_pct,socio_wgt,hh_wgt,minority_wgt,housing_wgt,overall_wgt,count,FIPS


In [ ]:
df_

In [141]:
df_rio = pd.DataFrame({'index':['Rio Arriba_NM'], 'socio_std':[0.094696], 'hh_std':[0.167799], 
                       'minority_std':[0.170754], 'housing_std':[0.178407],
       'overall_std':[0.099379], 'null_pct':[1.0], 'uninsured_pct':[0.012], 'socio_wgt':[0.69407], 
                       'hh_wgt':[0.740663],'minority_wgt':[0.68468], 'housing_wgt':[0.716337],
                       'overall_wgt':[0.781358], 'count':[9], 'FIPS':12345})

In [142]:
df_new = pd.concat([df_clean, df_rio])

In [143]:
index_dict = {'acadia_la':'acadia parish_la',
'aleutians east_ak':'aleutians east_ak',
'aleutians west_ak':'aleutians west census area_ak',
'allen_la':'allen parish_la',
'anchorage_ak':'anchorage municipality_ak',
'ascension_la':'ascension parish_la',
'assumption_la':'assumption parish_la',
'avoyelles_la':'avoyelles parish_la',
'beauregard_la':'beauregard parish_la',
'bethel_ak':'bethel census area_ak',
'bienville_la':'bienville parish_la',
'bossier_la':'bossier parish_la',
'bristol bay_ak':'bristol bay borough_ak',
'caddo_la':'caddo parish_la',
'calcasieu_la':'calcasieu parish_la',
'caldwell_la':'caldwell parish_la',
'cameron_la':'cameron parish_la',
'catahoula_la':'catahoula parish_la',
'claiborne_la':'claiborne parish_la',
'concordia_la':'concordia parish_la',
'de soto_la':'de soto parish_la',
'denali_ak':'denali borough_ak',
'dillingham_ak':'dillingham census area_ak',
'do√±a ana_nm':'dona ana_nm',
'east baton rouge_la':'east baton rouge parish_la',
'east carroll_la':'east carroll parish_la',
'east feliciana_la':'east feliciana parish_la',
'evangeline_la':'evangeline parish_la',
'fairbanks north star_ak':'fairbanks north star borough_ak',
'franklin_la':'franklin parish_la',
'grant_la':'grant parish_la',
'haines_ak':'haines borough_ak',
'hoonah-angoon_ak':'hoonah-angoon census area_ak',
'iberia_la':'iberia parish_la',
'iberville_la':'iberville parish_la',
'jackson_la':'jackson parish_la',
'jefferson davis_la':'jefferson davis parish_la',
'jefferson_la':'jefferson parish_la',
'kenai peninsula_ak':'kenai peninsula borough_ak',
'ketchikan gateway_ak':'ketchikan gateway borough_ak',
'kodiak island_ak':'kodiak island borough_ak',
'kusilvak_ak':'kusilvak census area_ak',
'lafayette_la':'lafayette parish_la',
'lafourche_la':'lafourche parish_la',
'lake and peninsula_ak':'lake and peninsula borough_ak',
'lincoln_la':'lincoln parish_la',
'livingston_la':'livingston parish_la',
'madison_la':'madison parish_la',
'matanuska-susitna_ak':'matanuska-susitna borough_ak',
'morehouse_la':'morehouse parish_la',
'natchitoches_la':'natchitoches parish_la',
'nome_ak':'nome census area_ak',
'north slope_ak':'north slope borough_ak',
'northwest arctic_ak':'northwest arctic borough_ak',
'orleans_la':'orleans parish_la',
'ouachita_la':'ouachita parish_la',
'petersburg_ak':'petersburg borough_ak',
'plaquemines_la':'plaquemines parish_la',
'pointe coupee_la':'pointe coupee parish_la',
'prince of wales-hyder_ak':'prince of wales-hyder census area_ak',
'rapides_la':'rapides parish_la',
'red river_la':'red river parish_la',
'richland_la':'richland parish_la',
'sabine_la':'sabine parish_la',
'skagway_ak':'skagway municipality_ak',
'southeast fairbanks_ak':'southeast fairbanks census area_ak',
'st. bernard_la':'st. bernard parish_la',
'st. charles_la':'st. charles parish_la',
'st. helena_la':'st. helena parish_la',
'st. james_la':'st. james parish_la',
'st. john the baptist_la':'st. john the baptist parish_la',
'st. landry_la':'st. landry parish_la',
'st. martin_la':'st. martin parish_la',
'st. mary_la':'st. mary parish_la',
'st. tammany_la':'st. tammany parish_la',
'tangipahoa_la':'tangipahoa parish_la',
'tensas_la':'tensas parish_la',
'terrebonne_la':'terrebonne parish_la',
'union_la':'union parish_la',
'vermilion_la':'vermilion parish_la',
'vernon_la':'vernon parish_la',
'washington_la':'washington parish_la',
'webster_la':'webster parish_la',
'west baton rouge_la':'west baton rouge parish_la',
'west carroll_la':'west carroll parish_la',
'west feliciana_la':'west feliciana parish_la',
'winn_la':'winn parish_la',
'yukon-koyukuk_ak':'yukon-koyukuk census area_ak',
'aleutians east_ak':'aleutians east borough_ak',
'lasalle_la':'la salle parish_la'}

In [144]:
df_new['index'] = df_new['index'].str.lower()
df_new.replace({'index':index_dict}, inplace=True)

In [145]:
df_new.set_index('index', inplace=True)

In [146]:
df_new.to_csv('../clean_data/svi_clean.csv')

In [147]:
df_new.shape

(3142, 13)